In [1]:
import pandas as pd
import numpy as np
#pip install python-coinmarketcap
import coinmarketcapapi
import yfinance as yf
from tqdm import tqdm

#use your API KEY that you get here: https://coinmarketcap.com/api/
cmc = coinmarketcapapi.CoinMarketCapAPI('YOUR_API')

## Get marketcap and change data from CoinMarketCap

In [24]:
#Obtain a list of 100 biggest active cryptocurrencies with latest market data
data_listing = cmc.cryptocurrency_listings_latest()
data_listing.data[0]

{'id': 1,
 'name': 'Bitcoin',
 'symbol': 'BTC',
 'slug': 'bitcoin',
 'num_market_pairs': 9184,
 'date_added': '2013-04-28T00:00:00.000Z',
 'tags': ['mineable',
  'pow',
  'sha-256',
  'store-of-value',
  'state-channel',
  'coinbase-ventures-portfolio',
  'three-arrows-capital-portfolio',
  'polychain-capital-portfolio',
  'binance-labs-portfolio',
  'blockchain-capital-portfolio',
  'boostvc-portfolio',
  'cms-holdings-portfolio',
  'dcg-portfolio',
  'dragonfly-capital-portfolio',
  'electric-capital-portfolio',
  'fabric-ventures-portfolio',
  'framework-ventures-portfolio',
  'galaxy-digital-portfolio',
  'huobi-capital-portfolio',
  'alameda-research-portfolio',
  'a16z-portfolio',
  '1confirmation-portfolio',
  'winklevoss-capital-portfolio',
  'usv-portfolio',
  'placeholder-ventures-portfolio',
  'pantera-capital-portfolio',
  'multicoin-capital-portfolio',
  'paradigm-portfolio'],
 'max_supply': 21000000,
 'circulating_supply': 18967668,
 'total_supply': 18967668,
 'platform':

In [3]:
#convert to dataframe and select only needed columns
df = pd.DataFrame(data_listing.data)
df = df[['name', 'symbol', 'cmc_rank', 'last_updated', 'quote']]
df

,name,symbol,cmc_rank,last_updated,quote
0,Bitcoin,BTC,1,2022-02-25T04:55:00.000Z,"{'USD': {'price': 38761.88361368764, 'volume_2..."
1,Ethereum,ETH,2,2022-02-25T04:55:00.000Z,"{'USD': {'price': 2621.1053720709265, 'volume_..."
2,Tether,USDT,3,2022-02-25T04:55:00.000Z,"{'USD': {'price': 1.0004466980140234, 'volume_..."
3,BNB,BNB,4,2022-02-25T04:55:00.000Z,"{'USD': {'price': 363.62343002357784, 'volume_..."
4,USD Coin,USDC,5,2022-02-25T04:55:00.000Z,"{'USD': {'price': 1.0002169582693055, 'volume_..."
...,...,...,...,...,...
95,IoTeX,IOTX,96,2022-02-25T04:55:00.000Z,"{'USD': {'price': 0.07105788074312942, 'volume..."
96,renBTC,RENBTC,97,2022-02-25T04:55:00.000Z,"{'USD': {'price': 38591.74933447298, 'volume_2..."
97,XDC Network,XDC,98,2022-02-25T04:55:00.000Z,"{'USD': {'price': 0.049455368083038076, 'volum..."
98,Gnosis,GNO,99,2022-02-25T04:55:00.000Z,"{'USD': {'price': 320.3353915014257, 'volume_2..."


In [4]:
#data in quote column are in form of json, for easier manipulation split them in to individual columns and change names
quote = pd.json_normalize(df['quote']).iloc[:,:10]
quote.rename({'USD.price' : 'price',
              'USD.percent_change_24h' : '%_change_24h',
              'USD.percent_change_7d' : '%_change_7d',
              'USD.percent_change_30d' : '%_change_30d',
              'USD.percent_change_60d' : '%_change_60d',
              'USD.percent_change_90d' : '%_change_90d',
              'USD.market_cap' : 'market_cap',}, axis=1, inplace=True)
quote.drop(quote.columns[[1, 2, 3]], axis = 1, inplace = True)

quote.head()

,price,%_change_24h,%_change_7d,%_change_30d,%_change_60d,%_change_90d,market_cap
0,38761.883614,10.547319,-4.577206,2.783127,-24.055470,-28.931730,7.352225e+11
1,2621.105372,9.354793,-9.317384,5.287517,-35.621981,-36.705049,3.138020e+11
2,1.000447,-0.004411,-0.004706,0.009673,-0.034230,-0.090088,7.951901e+10
3,363.623430,6.825295,-9.979661,-4.664648,-33.740233,-38.775921,6.004032e+10
4,1.000217,0.065664,0.043600,0.041924,0.019535,-0.055794,5.320698e+10


In [5]:
#concatenate two dataframes in to one and change datetime format
data = pd.concat([df, quote], axis = 1, join = 'inner')
data = data.drop(columns = ['quote'])
#Drop rows containing USD (tether)
data.drop(data[data['symbol'].str.contains("US")].index.tolist(), inplace = True)
data['last_updated'] = pd.to_datetime(data['last_updated']).dt.strftime('%Y-%m-%d %H:%M')
data.head()

,name,symbol,cmc_rank,last_updated,price,%_change_24h,%_change_7d,%_change_30d,%_change_60d,%_change_90d,market_cap
0,Bitcoin,BTC,1,2022-02-25 04:55,38761.883614,10.547319,-4.577206,2.783127,-24.055470,-28.931730,7.352225e+11
1,Ethereum,ETH,2,2022-02-25 04:55,2621.105372,9.354793,-9.317384,5.287517,-35.621981,-36.705049,3.138020e+11
3,BNB,BNB,4,2022-02-25 04:55,363.623430,6.825295,-9.979661,-4.664648,-33.740233,-38.775921,6.004032e+10
5,XRP,XRP,6,2022-02-25 04:55,0.695486,7.101963,-11.328703,10.256083,-24.688628,-27.619566,3.334806e+10
6,Cardano,ADA,7,2022-02-25 04:55,0.850661,5.473493,-17.141730,-19.378999,-42.324816,-45.570413,2.861671e+10


In [6]:
#save as CSV file
data.to_csv('data.csv', index = False)

## Download OHLC data from Yahoofinance

In [7]:
symbols = data['symbol'].tolist()
usd_symbols = [i+'-USD' for i in symbols]

In [25]:
ohlc_data = {}
for i in tqdm(usd_symbols):
    try:
        ohlc_data[i[:-4]] = yf.download(i, period = '90D', progress=False)['Close']
    except:
        print('Problem with downloading data for:', i)

100%|███████████████████████████████████████████| 94/94 [00:16<00:00,  5.59it/s]


In [11]:
#Convert data in to Dataframe
ohlc_df_daily = pd.DataFrame(ohlc_data)
ohlc_df_daily = ohlc_df_daily.T
ohlc_df_daily['Symbol'] = symbols
ohlc_df_daily.head(5)

Date,2021-11-28 00:00:00,2021-11-29 00:00:00,2021-11-30 00:00:00,2021-12-01 00:00:00,2021-12-02 00:00:00,2021-12-03 00:00:00,2021-12-04 00:00:00,2021-12-05 00:00:00,2021-12-06 00:00:00,2021-12-07 00:00:00,...,2022-02-16 00:00:00,2022-02-17 00:00:00,2022-02-18 00:00:00,2022-02-19 00:00:00,2022-02-20 00:00:00,2022-02-21 00:00:00,2022-02-22 00:00:00,2022-02-23 00:00:00,2022-02-25 00:00:00,Symbol
BTC,57248.457031,57806.566406,57005.425781,57229.828125,56477.816406,53598.246094,49200.703125,49368.847656,50582.625000,50700.085938,...,43961.859375,40538.011719,40030.976562,40122.156250,38431.378906,37075.281250,38286.027344,37296.570312,38761.882812,BTC
ETH,4294.453613,4445.104980,4631.479004,4586.990234,4511.302246,4220.706055,4119.587402,4198.322754,4358.737305,4315.061523,...,3127.830078,2881.481934,2785.727539,2763.701172,2628.648438,2573.816162,2639.299316,2590.359619,2621.105469,ETH
BNB,611.366821,624.310059,622.669861,627.971558,619.470154,594.635986,569.077271,557.783081,589.256165,579.689880,...,427.918121,402.450043,399.569183,400.043579,380.825928,356.661255,374.307465,366.539490,363.623444,BNB
XRP,0.968751,0.991961,0.998754,0.990930,0.972312,0.922244,0.847029,0.805595,0.828288,0.818588,...,0.839036,0.767173,0.785890,0.822528,0.778296,0.703466,0.722833,0.698746,0.695486,XRP
ADA,1.592855,1.602508,1.554903,1.547713,1.715366,1.556555,1.420706,1.378105,1.425393,1.381941,...,1.084981,1.019895,0.997113,0.997703,0.932902,0.858236,0.889017,0.869169,0.850661,ADA


In [13]:
#Save as CSV file
ohlc_df_daily.to_csv('ohlc_df_daily.csv', index = False)